In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .appName("eda_retail_table") \
        .master("local[*]") \
        .config("spark.sql.shuffle.partitions", "8") \
        .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

# Data Transform and Clean

In [ ]:
retail_df = spark.read.csv("/content/online-retail-dataset.csv", header=True, inferSchema=True)

retail_df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [ ]:
retail_df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col, to_timestamp, sum, when, first

retail_df = retail_df.withColumn(
    "InvoiceNo",
    col("InvoiceNo").cast("INT"))


retail_df = retail_df.withColumn(
    "InvoiceDate",
    to_timestamp("InvoiceDate", "M/d/yyyy H:mm")
    )

In [ ]:
retail_df.printSchema()

root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:
retail_df.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [ ]:
retail_df.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in retail_df.columns
]).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|     9291|        0|       1454|       0|          0|        0|    135080|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [ ]:
retail_df.filter(col('CustomerID').isNull()).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536414|    22139|                NULL|      56|2010-12-01 11:52:00|      0.0|      NULL|United Kingdom|
|   536544|    21773|DECORATIVE ROSE B...|       1|2010-12-01 14:32:00|     2.51|      NULL|United Kingdom|
|   536544|    21774|DECORATIVE CATS B...|       2|2010-12-01 14:32:00|     2.51|      NULL|United Kingdom|
|   536544|    21786|  POLKADOT RAIN HAT |       4|2010-12-01 14:32:00|     0.85|      NULL|United Kingdom|
|   536544|    21787|RAIN PONCHO RETRO...|       2|2010-12-01 14:32:00|     1.66|      NULL|United Kingdom|
|   536544|    21790|  VINTAGE SNAP CARDS|       9|2010-12-01 14:32:00|     1.66|      NULL|United Kingdom|
|   536544|    21791|VINTAGE

In [ ]:
retail_df = retail_df.filter(col("StockCode").rlike("^[0-9]+[A-Z]?$"))

In [ ]:
retail_df.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in retail_df.columns
]).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|     8699|        0|       1439|       0|          0|        0|    132031|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [ ]:
retail_df = retail_df.fillna({"InvoiceNo": 0, "CustomerID" : 0, "Description" : "None temp"})

In [ ]:
retail_df.select(['StockCode']).distinct().count()

3923

In [ ]:
retail_df.select(['Description']).distinct().count()

4196

In [ ]:
# description_lookup = retail_df.filter(col("Description").isNotNull()) \
#     .groupBy("StockCode") \
#     .agg(first("Description", ignorenulls=True).alias("Description_filled"))

# retail_df = retail_df.join(description_lookup, on="StockCode", how="left")

# retail_df = retail_df.withColumn(
#     "Description",
#     when(col("Description").isNull(), col("Description_filled"))
#     .otherwise(col("Description"))
# )

# retail_df = retail_df.drop("Description_filled")

In [ ]:
retail_df.sort('Quantity', ascending = True).show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|        0|    23843|PAPER CRAFT , LIT...|  -80995|2011-12-09 09:27:00|     2.08|     16446|United Kingdom|
|        0|    23166|MEDIUM CERAMIC TO...|  -74215|2011-01-18 10:17:00|     1.04|     12346|United Kingdom|
|   556690|    23005|printing smudges/...|   -9600|2011-06-14 10:37:00|      0.0|         0|United Kingdom|
|   556691|    23005|printing smudges/...|   -9600|2011-06-14 10:37:00|      0.0|         0|United Kingdom|
|        0|    84347|ROTATING SILVER A...|   -9360|2010-12-02 14:23:00|     0.03|     15838|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [ ]:
retail_df = retail_df.filter((col("Quantity") > 0) & (col("UnitPrice") > 0)).sort('InvoiceDate', ascending = True)

In [ ]:
retail_df.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in retail_df.columns
]).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|          0|       0|          0|        0|         0|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [ ]:
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, hour, quarter

retail_df = retail_df.withColumn("Year", year("InvoiceDate")) \
    .withColumn("Month", month("InvoiceDate")) \
    .withColumn("Day", dayofmonth("InvoiceDate")) \
    .withColumn("DayOfWeek", dayofweek("InvoiceDate")) \
    .withColumn("Hour", hour("InvoiceDate")) \
    .withColumn("Quarter", quarter("InvoiceDate"))

retail_df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----+-----+---+---------+----+-------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|Year|Month|Day|DayOfWeek|Hour|Quarter|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----+-----+---+---------+----+-------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|     17850|United Kingdom|2010|   12|  1|        4|   8|      4|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|2010|   12|  1|        4|   8|      4|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|     17850|United Kingdom|2010|   12|  1|        4|   8|      4|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|2010|   12|  1|  

In [ ]:
retail_df.sort('Quantity', ascending = False).show(5, truncate = False)

+---------+---------+---------------------------------+--------+-------------------+---------+----------+--------------+----+-----+---+---------+----+-------+
|InvoiceNo|StockCode|Description                      |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |Year|Month|Day|DayOfWeek|Hour|Quarter|
+---------+---------+---------------------------------+--------+-------------------+---------+----------+--------------+----+-----+---+---------+----+-------+
|581483   |23843    |PAPER CRAFT , LITTLE BIRDIE      |80995   |2011-12-09 09:15:00|2.08     |16446     |United Kingdom|2011|12   |9  |6        |9   |4      |
|541431   |23166    |MEDIUM CERAMIC TOP STORAGE JAR   |74215   |2011-01-18 10:01:00|1.04     |12346     |United Kingdom|2011|1    |18 |3        |10  |1      |
|573008   |84077    |WORLD WAR 2 GLIDERS ASSTD DESIGNS|4800    |2011-10-27 12:26:00|0.21     |12901     |United Kingdom|2011|10   |27 |5        |12  |4      |
|554868   |22197    |SMALL POPCORN HOLDER     

# Data Analysis

## Use Case: Customer Segmentation using RFM Analysis

In [ ]:
retail_df.createOrReplaceTempView("retail")

In [ ]:
query = """
WITH max_date AS (
  SELECT MAX(InvoiceDate) AS max_date FROM retail
),
rfm AS (
  SELECT
    CustomerID,
    DATEDIFF((SELECT max_date FROM max_date), MAX(InvoiceDate)) AS Recency,
    COUNT(DISTINCT InvoiceNo) AS Frequency,
    ROUND(SUM(Quantity * UnitPrice), 2) AS Monetary
  FROM retail
  GROUP BY CustomerID
),
scored_rfm AS (
  SELECT *,
    CASE
      WHEN Recency <= 30 THEN 3
      WHEN Recency <= 90 THEN 2
      ELSE 1
    END AS R_Score,
    CASE
      WHEN Frequency >= 10 THEN 3
      WHEN Frequency >= 5 THEN 2
      ELSE 1
    END AS F_Score,
    CASE
      WHEN Monetary >= 1000 THEN 3
      WHEN Monetary >= 500 THEN 2
      ELSE 1
    END AS M_Score
  FROM rfm
)
SELECT *,
       R_Score + F_Score + M_Score AS RFM_Score,
       CASE
         WHEN (R_Score + F_Score + M_Score) >= 8 THEN 'High Value'
         WHEN (R_Score + F_Score + M_Score) >= 5 THEN 'Mid Value'
         ELSE 'Low Value'
       END AS Segment
FROM scored_rfm
"""
rfm_df = spark.sql(query)
rfm_df.show()
rfm_pdf = rfm_df.toPandas()
rfm_pdf.to_csv("rfm_customer_data.csv", index=False)

+----------+-------+---------+--------+-------+-------+-------+---------+----------+
|CustomerID|Recency|Frequency|Monetary|R_Score|F_Score|M_Score|RFM_Score|   Segment|
+----------+-------+---------+--------+-------+-------+-------+---------+----------+
|     17850|    372|       34| 5272.41|      1|      3|      3|        7| Mid Value|
|     12915|    148|        2|  363.65|      1|      1|      1|        3| Low Value|
|     17802|     82|        3| 1285.04|      2|      1|      3|        6| Mid Value|
|     16814|     14|       21|  6487.3|      3|      3|      3|        9|High Value|
|     15018|     38|        3|  496.21|      2|      1|      1|        4| Low Value|
|     15024|      9|       10| 1661.33|      3|      3|      3|        9|High Value|
|     15021|      8|        9| 1902.92|      3|      2|      3|        8|High Value|
|     13777|      0|       33|25525.21|      3|      3|      3|        9|High Value|
|     13090|      8|       13| 8895.82|      3|      3|      3|  